# Demo: Implementing Dynamic Data Masking (M04\_L04\_Demo01)

**<span style="color:black;mso-color-alt:windowtext">Objective<i>:&nbsp; </i></span>** <span style="color:black;
mso-color-alt:windowtext">The goal of this demo is to demonstrate how to implement Dynamic Data Masking in Azure Synapse SQL Pool.&nbsp;&nbsp;</span>

In [11]:
CREATE TABLE dbo.DDMExample 
(
    UserID	INT IDENTITY(1,1) PRIMARY KEY NONCLUSTERED NOT ENFORCED,
    Firstname VARCHAR(40), 
    Lastname VARCHAR(40), 
    Username VARCHAR(40), 
    UserLoginID bigint MASKED WITH (FUNCTION = 'random(50000, 75000)'), --Notice the Mask functions
    Email VARCHAR(50) MASKED WITH (FUNCTION = 'email()'), 
    UserPwd VARCHAR(50) MASKED WITH (FUNCTION = 'default()')
); 
GO

Commands completed successfully.

Total execution time: 00:00:00.616

In [12]:
--Insert a couple of records to use as an example
INSERT INTO dbo.DDMExample (Firstname, Lastname, Username, UserLoginID, Email, UserPwd) 
VALUES ('John','Smith','JSmith', 372036854775808, 'johnsmith@gmail.com','123456ABCDE');
 
INSERT INTO dbo.DDMExample (Firstname, Lastname, Username, UserLoginID, Email, UserPwd) 
VALUES ('Jane','Doe','JDoe', 372032254855106, 'janedoe@gmail.com','112233ZYXWV');
 
INSERT INTO dbo.DDMExample (Firstname, Lastname, Username, UserLoginID, Email, UserPwd) 
VALUES ('Walt','Disney','WDisney', 372031114679991, 'waltdisney@gmail.com','998877AZBYC'); 

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.251

In [3]:
--2. Create a test user for Execute AS statements
CREATE USER TestUser WITHOUT LOGIN; 
GO 
GRANT SELECT ON dbo.DDMExample TO TestUser; 
GO  

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.145

In [4]:
--3. Query the table as Test user - Queries executed the as the TestUser view masked data. 
EXECUTE AS USER = 'TestUser';   
SELECT * FROM dbo.DDMExample ;   
REVERT;    
GO 

(3 rows affected)

Total execution time: 00:00:00.397

UserID,Firstname,Lastname,Username,UserLoginID,Email,UserPwd
3,Walt,Disney,WDisney,57858,wXXX@XXXX.com,xxxx
26,Jane,Doe,JDoe,73828,jXXX@XXXX.com,xxxx
37,John,Smith,JSmith,50352,jXXX@XXXX.com,xxxx


In [5]:
--4. Remove masking on email column
ALTER TABLE dbo.DDMExample ALTER COLUMN [Email] DROP MASKED

Commands completed successfully.

Total execution time: 00:00:00.109

In [6]:
--5. Run the select against after maksing has been removed from email and know you will be able to see complete email address

EXECUTE AS USER = 'TestUser';   
SELECT * FROM dbo.DDMExample ;   
REVERT;    
GO 

(3 rows affected)

Total execution time: 00:00:00.239

UserID,Firstname,Lastname,Username,UserLoginID,Email,UserPwd
3,Walt,Disney,WDisney,71439,waltdisney@gmail.com,xxxx
26,Jane,Doe,JDoe,58687,janedoe@gmail.com,xxxx
37,John,Smith,JSmith,60122,johnsmith@gmail.com,xxxx


In [7]:
--6. Remove masking on Password column
ALTER TABLE dbo.DDMExample ALTER COLUMN [UserPwd] DROP MASKED

Commands completed successfully.

Total execution time: 00:00:00.091

In [ ]:
--7. Run the select against after masking has been removed from Password column and know you will be able to see complete the complete Pasword

EXECUTE AS USER = 'TestUser';   
SELECT * FROM dbo.DDMExample ;   
REVERT;    
GO 

In [8]:
-- 8.  Remove masking on UserID column
ALTER TABLE dbo.DDMExample ALTER COLUMN [UserLoginID] DROP MASKED

Commands completed successfully.

Total execution time: 00:00:00.101

In [13]:

--9. Run the select against after masking has been removed from User ID column and know you will be able to see the complete USER ID
--EXECUTE AS USER = 'TestUser';   
SELECT * FROM dbo.DDMExample ;   
REVERT;    
GO 

(3 rows affected)

Total execution time: 00:00:00.279

UserID,Firstname,Lastname,Username,UserLoginID,Email,UserPwd
27,Walt,Disney,WDisney,372031114679991,waltdisney@gmail.com,998877AZBYC
35,John,Smith,JSmith,372036854775808,johnsmith@gmail.com,123456ABCDE
52,Jane,Doe,JDoe,372032254855106,janedoe@gmail.com,112233ZYXWV


In [10]:
--10. Cleanup
DROP USER TestUser
DROP TABLE DDMExample



Commands completed successfully.

Total execution time: 00:00:00.309